# ZMS Lab4 Gwintex - Julia

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Opis-zajęć" data-toc-modified-id="Opis-zajęć-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Opis zajęć</a></span></li><li><span><a href="#ROZWIĄZANIE" data-toc-modified-id="ROZWIĄZANIE-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>ROZWIĄZANIE</a></span><ul class="toc-item"><li><span><a href="#Stałe-i-zmienne-wykorzystane-w-modelu" data-toc-modified-id="Stałe-i-zmienne-wykorzystane-w-modelu-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Stałe i zmienne wykorzystane w modelu</a></span></li><li><span><a href="#Model" data-toc-modified-id="Model-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Model</a></span></li><li><span><a href="#Funkcja-do-uruchomienia-symulacji" data-toc-modified-id="Funkcja-do-uruchomienia-symulacji-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Funkcja do uruchomienia symulacji</a></span></li><li><span><a href="#Symulacja" data-toc-modified-id="Symulacja-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Symulacja</a></span></li></ul></li><li><span><a href="#Raport---zadanie-domowe" data-toc-modified-id="Raport---zadanie-domowe-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Raport - zadanie domowe</a></span></li></ul></div>

## Opis zajęć


Firma GWINTEX S.A. jest międzynarodowym potentatem w dziedzinie produkcji korkociągów. Korkociągi są wytwarzane na bardzo nowoczesnych maszynach metalurgicznych. W związku ze znacznym wzrostem zamówień firma planuje uruchomienie nowej hali produkcyjnej, w której znajdzie się **n=6 maszyn**. Do każdej maszyny jest przypisany operator, który jest odpowiedzialny za jej obsługę oraz usuwanie awarii. Na podstawie pomiarów historycznych wiadomo, że **czas bezawaryjnej pracy maszyny ma rozkład wykładniczy ze średnią 75 minut**. W przypadku wystąpienia awarii operator dzwoni do warsztatu z prośbą o dostarczenie pakietu narzędzi naprawczych. Pakiet narzędzi jest bardzo ciężki i w związku z tym musi być transportowany za pomocą przenośnika taśmowego (taśmociągu). **Czas transportu zestawu narzędzi do maszyny wynosi *ti*, i=1..6. Czas naprawy jest zmienną losową z rozkładu Erlanga k=3 i średnio wynosi 15 minut**. Po ukończeniu naprawy narzędzia są powtórnie umieszczane na taśmociągu i wracają w komplecie do warsztatu celem ich uzupełnienia. Ze względu na specyfikę specjalistycznych narzędzi nie jest możliwe dokonywanie kolejnych napraw przed powrotem narzędzi do warsztatu. Ze względu na bardzo wysoką cenę jednego pakietu narzędzi naprawczych ich liczba ***m* jest mniejsza od liczby maszyn w hali produkcyjnej**. Gdy w danej chwili pakiet narzędzi nie jest dostępny operator czeka aż inny pakiet wróci do warsztatu i zostanie mu wysłany.

Zarząd firmy GWINTEX zastanawia się **jakie powinno być rozmieszczenie urządzeń na hali produkcyjnej** oraz **ile pakietów narzędziowych do obsługi maszyn należy zakupić**. Rozważane są dwie organizacje hali produkcyjnej – układ liniowy oraz układ gniazdowy. **W układzie liniowym czas transportu narzędzi z warsztatu do maszyny wynosi ti=2*i**, natomiast **w układzie gniazdowym czas ten jest stały i wynosi 3 minuty**. Czas transportu narzędzi do warsztatu jest taki sam jak czas transportu do maszyny. Wprowadzenie układu gniazdowego wiąże się z wyższymi kosztami instalacyjnymi związanymi z uruchomieniem sześciu niezależnych taśmociągów.


## ROZWIĄZANIE

### Stałe i zmienne wykorzystane w modelu

In [ ]:
# liczba maszyn
n = 6 

# średni czas pracy bez usterki
avg_working_time = 75. # minut

# średni czas naprawy
avg_repair_time = 15. # minut

# ilość zestawów narzędzi
m = 6

# horyzont analizy
horizon = 30 # dni

# liczba uruchomień symulacji
iterations = 10   

### Model

Wektory, które mają za zadanie kontrolować stan symulacji:

- momenty wystąpienia kolejnych zdarzeń
- status narzędzi i maszyn 
    - `W` - pracuje 
    - `Q` - czeka na narzedzia 
    - `R` - jest naprawiona
- czas ich bezczynności
- events --> wektor zdarzeń, które zmieniają stan symulacji (np. zepsucie się maszyny, czas naprawy, itp.)

In [ ]:
using Distributions

In [ ]:
function model(horizon::Int, avg_working_time::Float64, avg_repair_time::Float64, n::Int, m::Int, setup::String)
    # setup - układ liniowy "L" lub gniazdowy "G"
    
    # horyzont działania w minutach
    horizon = horizon * 24 * 60 
    
    # wektor zdarzeń, który zmienia stan symulacji
    #?Exponential()
    events = rand(Exponential(avg_working_time), n)
    
    # status - określa aktualny stan maszyny 
    status = fill("W", n)

    # t_start - określa początek bezczynności maszyny
    t_start = zeros(n)

    # t_cum - skumulowany czas bezczynności maszyny
    t_cum = fill(0., n)

    # tools_loc lokalizacja narzedzi - albo numer maszyny albo -1 czyli warsztat
    tools_loc = repeat([-1], m)

    # tools_occupied czas zajecia zestawu przez naprawianą maszynę
    tools_occupied = zeros(m)
    
    # zegar symulacji- najblizsze zadanie, które ma być wykonane
    t = minimum(events)
    
    # rozpoczynamy symulacje "skacząc" po kolejnych zdarzeniach  
    while t <= horizon   
        # jeżeli zestawy nie są aktualnie zajęte to przenosimy je z powrotem do warsztatu
        tools_loc[tools_occupied .<= t] .= -1

        # wybieramy maszynę, której dotyczy zdarzenie
        machine = findfirst(==(t),events)
        
        """
        Gdy maszyna, której dotyczy zdarzenie ma status "W"
            - to najpierw zaktualizuj wektor t_start dla tej maszyny jako początek jej bezczynności = t.
            - następnie sprawdź czy dostępny jest jakiś zestaw naprawczy. Jezeli nie
                - to ustaw status maszyny na "Q" 
                - zaktualizuj wektor events podajac mu najkrótszy czas oczekiwania na wolny zestaw.
              Jeżeli tak
                - ustaw status maszyny na "R"
                - wyznacz czas  potrzebny na naprawę maszyny w zależności od ukladu taśmociągu 
                (czas transportu + czas naprawy)
                - ustaw koniec naprawy jako zdarzenie dla danej maszyny
                - zaktualizuj wektor tools_loc dla odpowiedniego zestawu podając numer maszyny, którą on obsługuje
                - zaktualizuj wektor tools_occupied czasem jaki mu to zajmie (2* transport + naprawa)
        """
        if status[machine] == "W"
            t_start[machine] = t
            tools = findfirst(x->x==-1, tools_loc)
            if typeof(tools) == Nothing
                status[machine] = "Q"
                events[machine] = minimum(tools_occupied)
            else
                status[machine] = "R"
                if setup == "L"
                    transport_time = 2 * machine
                elseif setup == "G"
                    transport_time =  3
                else
                    throw(ArgumentError("Niepoprawny układ maszyn! Należy wybrać układ 'L' lub 'G'!"))
                end          
                repair_time = rand(Gamma(3, avg_repair_time/3))
                #?Gamma
                events[machine] += repair_time + transport_time
                tools_loc[tools] = machine
                tools_occupied[tools] = t + repair_time + 2 * transport_time
                
                """
                Gdy maszyna ma status "Q"
                    - wybierz dostępny zestaw naprawczy
                    - ustal status maszyny na "R"
                    - zaktualizuj wektor tools_loc lokalizacją narzedzi i tools_occupied 
                    czasem jaki zajmie ich transport (w dwie strony) i naprawa maszyny
                    -zaktualizuj wektor zdarzeń czasem potrzebnym na naprawę maszyny i transport narzedzi
                """
            end 
         elseif status[machine] == "Q"
            tools = findfirst(x->x==-1, tools_loc)
            if typeof(tools) == Nothing
                status[machine] = "Q"
                events[machine] = minimum(tools_occupied)
            else
                status[machine] = "R"
                if setup == "L"
                    transport_time = 2 * machine
                elseif setup == "G"
                    transport_time =  3
                else
                    throw(ArgumentError("Niepoprawny układ maszyn! Należy wybrać układ 'L' lub 'G'!"))
                end
                repair_time = rand(Gamma(3, avg_repair_time/3))
                events[machine] += repair_time + transport_time
                tools_loc[tools] = machine
                tools_occupied[tools] = t + repair_time + 2 * transport_time
            end
            """
            Gdy maszyna ma status "R"
                - ustal jej status na "W"
                - wyznacz czas kolejnej awarii i zaktualizuj wektor events
                - wylicz czas bezczynnosci i uzupelnij o niego liste t_cum
            """
            
        else
            status[machine] = "W"
            events[machine] += rand(Exponential(avg_working_time))
            t_cum[machine] += t - t_start[machine]
        end
        # ustalamy nowe t
        t = minimum(events)
    end   
    # wynik - liste skumulowanych bezczynnosci dla kazdej z maszyn
    return t_cum
end

### Funkcja do uruchomienia symulacji

In [ ]:
function run_model(iterations::Int, horizon, avg_working_time, avg_repair_time, n, m, setup)
    avg_t_cum = [model(horizon, avg_working_time, avg_repair_time, n, m, setup) for i in 1:iterations]
    return mean.(avg_t_cum)
end

### Symulacja

In [ ]:
runL = run_model(iterations, horizon, avg_working_time, avg_repair_time, n, m, "L")

## Raport - zadanie domowe

Należy wprowadzić do modelu **3 czynniki kosztów**:
1. Koszt przestoju maszyny (koszt jednostkowy np. za minutę przestoju)
2. Dodatkowy koszt instalacji w ustawieniu gniazdowym ("G") w porównaniu do instalcji liniowej ("L")
3. Koszt zestawu narzędzi

---

Po modyfikacji kodu należy odpowiedzieć na pytania:
* Jak zmienia się optymalna liczba zestawów narzędziowych w zależności od kosztów przestoju i kosztów zestawu narzędzi w przypadku a) układu liniowego b) układu gniazdowego (wielowymiarowa analiza (koszt przestoju) x (koszt narzędzi) przy stałym układzie produkcyjnym)?
* Który układ produkcyjny jest lepszy w zależności od kosztów przestoju i różnicy między kosztem instalacji "G" i "L" (proszę przyjąć m=6)?
* Jak zmienia się rozwiązanie optymalne o możliwych wartościach "G1, G2,...,G6" lub "L1, L2, ..., L6" (pierwszy znak - układ produkcyjny, drugi znak - liczba zestawów narzędzi max.6) w zależności od wszystkich 3 źródeł kosztów podanych powyżej. Disclaimer: nie wszystkie z podanych rozwiązań muszą być rozwiązaniami optymalnymi.

Celem jest minimalizacja kosztów. Liczbę maszyn, średni czas pracy bez usterki, średni czas naprawy oraz horyzont symulacji proszę pozostawić jak w przykładzie. W wynikach analiz proszę pamiętać, aby obok średniej zawsze pojawiała się jakaś miara niepewności np. odchylenie standardowe (na wykresach słupki błędów/przedziały ufności jeśli jest to możliwe). Mogą Państwo rozbudować analizę według uznania.

---
Pliki:
* **Wersja Jupyter** : 
    1. Notebook w formacie `.ipynb` 
    2. Plik `.html/.pdf` wygenerowany z notebooka
* **Wersja Word**: 
     1. Skrypt z kodem w formacie `.py/.jl`, 
     2. Plik Worda w formacie`.docx`, 
     3. Inne pliki użyte w trakcie pracy np. `.xlsx`, 
     4. Plik `.pdf` wygnerowany z pliku Word

Pliki raportu proszę spakować do pliku `.zip` o nazwie __ZMS202021L_R2_nazwisko1_nazwisko2_nazwisko3_nazwisko4.zip__ i wgrać na Moodle

Struktura raportu: http://moodle.szufel.pl/mod/page/view.php?id=61 (sekcja **Zawartość raportu**)

**Język raportu**: Python lub Julia

